In [13]:
from trism import TritonModel
import numpy as np

# Using the ensemble model (recommended)
ensemble = TritonModel(
    model="ensemble_model",
    version=1,
    url="localhost:7000",
    grpc=False
)

# Process text input
input_text = ["Tôi đang học AI"]
input_text = [text.encode("utf-8") for text in input_text]  # Encode to bytes
input_text = np.array(input_text, dtype=object)  # Ensure input is a numpy array
input_text = np.expand_dims(input_text, axis=0)  # Add batch dimension

result = ensemble.run(data=[input_text])
embeddings = result["embedding"]
print("Embeddings shape:", embeddings.shape)

# Using individual models
tokenizer = TritonModel(model="tokenizer", version=1, url="localhost:7001")
ctx_encoder = TritonModel(model="retrieve", version=1, url="localhost:7001")

# Tokenize
tokens = tokenizer.run(data=[input_text])
# Encode
embeddings_2 = ctx_encoder.run(data=[
    tokens["input_ids"],
    tokens["attention_mask"], 
    tokens["token_type_ids"]
])['last_hidden_state']
print("Embeddings shape:", embeddings_2.shape)

# check if embeddings match
np.allclose(embeddings, embeddings_2, rtol=1e-05, atol=1e-08)

Embeddings shape: (1, 128, 768)
Embeddings shape: (1, 128, 768)


True